In [2]:
!pip install lightgbm
# !python3 -m pip install --upgrade pip


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.3 MB/s eta 0:00:00-:--:--
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [20]:
import pandas as pd
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report

In [72]:
train_df = pd.read_csv(".././Training-Testing Dataset/Training Data (RWS Oversampling).csv")
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data (RWS Oversampling).csv")
target_col = "target"

X_train = train_df.drop(columns=[target_col])
X_test = test_df.drop(columns=[target_col])

y_train = train_df[target_col]
y_test  = test_df[target_col]

print(y_train.value_counts())
categorical_columns = [col for col in X_train.columns if X_train[col].dtype == 'object']

for col in categorical_columns:
    X_train[col] = X_train[col].astype("category")
    X_test[col] = X_test[col].astype("category")

#LGBM only accepts numerical data in the target columns
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

train_set = lgbm.Dataset(X_train,label=y_train,categorical_feature=categorical_columns)
test_set = lgbm.Dataset(X_test,label=y_test,categorical_feature=categorical_columns)


target
Medium_No     4940
High_No       4219
Low_No        3872
Medium_Yes    3347
High_Yes      3266
Low_Yes       3220
Name: count, dtype: int64


In [23]:
print(len(set(y_train)))

6


In [75]:
params = {
    'objective':'multiclass',
    'num_class':len(set(y_train)),
    'metric':'multi_error',
    'learning_rate':0.09,
    'num_leaves':70,
    'max_depth':-1,
    'verbose':-1
}

model = lgbm.train(params,train_set,valid_sets=[test_set],num_boost_round=500)

In [76]:
# Predict
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.9456106870229007
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3144
           1       0.98      0.98      0.98      3144
           2       0.92      0.95      0.94      3144
           3       0.96      1.00      0.98      3144
           4       0.93      0.85      0.89      3144
           5       0.96      0.99      0.98      3144

    accuracy                           0.95     18864
   macro avg       0.95      0.95      0.94     18864
weighted avg       0.95      0.95      0.94     18864



In [79]:
test_df = pd.read_csv(".././Training-Testing Dataset/Training Data (CC undersampling).csv")

target_col = "target"

X_test = test_df.drop(columns=[target_col])

y_test  = test_df[target_col]

print(y_test.value_counts())

for col in categorical_columns:
    X_test[col] = X_test[col].astype("category")

y_test = le.fit_transform(y_test)


target
High_No       132
High_Yes      132
Low_No        132
Low_Yes       132
Medium_No     132
Medium_Yes    132
Name: count, dtype: int64


In [78]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.5138888888888888
              precision    recall  f1-score   support

           0       0.27      0.33      0.29       132
           1       0.93      0.75      0.83       132
           2       0.32      0.20      0.24       132
           3       0.90      0.81      0.85       132
           4       0.27      0.52      0.36       132
           5       0.91      0.48      0.63       132

    accuracy                           0.51       792
   macro avg       0.60      0.51      0.53       792
weighted avg       0.60      0.51      0.53       792

